# Importing packets

In [1]:
import pandas as pd
import re
import numpy as np
import networkx as nx
from functools import reduce

# Funtion to get centrality measures

In [45]:
#calculating the in_degree #not weighted 

def network_measure(edgelist_object):
    colname = edgelist_object.columns[1]
    
    G_dir = nx.from_pandas_edgelist(edgelist_object, 'actor', colname, create_using=nx.DiGraph())
    G_dir_undirected = nx.from_pandas_edgelist(edgelist_object, 'actor', colname)
    
    #in-degree
    in_degree = dict(G_dir.in_degree())
    df_in_degree = pd.DataFrame(list(in_degree.items()),
                                columns = ['actor','in_degree']).sort_values(by='actor', ascending=True)
    #out-degree
    out_degree = dict(G_dir.out_degree())
    df_out_degree = pd.DataFrame(list(out_degree.items()),
                                 columns = ['actor','out_degree']).sort_values(by='actor', ascending=True)

    #closeness centrality
    closeness = dict(nx.closeness_centrality(G_dir_undirected))
    df_closeness_degree = pd.DataFrame(list(closeness.items()),
                                       columns = ['actor', 'closeness']).sort_values(by = 'actor', ascending=True)
    #betweeness centrality
    betweeness = dict(nx.betweenness_centrality(G_dir_undirected))
    df_betweeness = pd.DataFrame(list(betweeness.items()),
                                       columns = ['actor', 'betweeness']).sort_values(by = 'actor', ascending=True)

    dfs = [df_in_degree, df_out_degree, df_closeness_degree, df_betweeness]
    df_network_measures = reduce(lambda left,right: pd.merge(left,right,on='actor'), dfs)
    return df_network_measures

# SNA - Retweet network

## Importing data

In [2]:
#read data
data_retweets = pd.read_csv('data_retweets.csv')
data_retweets = data_retweets.drop(['@mentions'], axis=1) # We start by dropping the column about mentions.

print(data_retweets.shape)
data_retweets.head(3)

(5465, 2)


,actor,retweet
0,PlanBornefonden,dorthe10
1,PlanBornefonden,dorthe10
2,PlanBornefonden,anne_smith_p


## Creating retweet network

In [3]:
retweet_dict = {} 

#for actor, retweet in zip(data_retweets.actor, data_retweets.retweet):
for pair in zip(data_retweets['actor'], data_retweets['retweet']):
    
    if pair not in retweet_dict:
        retweet_dict[pair] = 1
    else:
        retweet_dict[pair] += 1


In [4]:
edgelist_df = pd.DataFrame(columns=["actor", "retweet", "weight"])

actors = []
retweets = []
weights = []

for actor, retweet in retweet_dict:
    weights.append(retweet_dict[(actor, retweet)])
    actors.append(actor)
    retweets.append(retweet)
    

In [5]:
edgelist_df['actor'], edgelist_df["retweet"], edgelist_df['weight'] = actors, retweets, weights
edgelist_df = edgelist_df.dropna()

In [6]:
edgelist_df

,actor,retweet,weight
0,PlanBornefonden,dorthe10,67
1,PlanBornefonden,anne_smith_p,61
2,PlanBornefonden,SeeRap,1
3,PlanBornefonden,UNFPAEthiopia,1
4,PlanBornefonden,udviklingsmin,8
...,...,...,...
1213,CaritasDanmark,Caritaslb,1
1214,CaritasDanmark,Pontifex,2
1215,CaritasDanmark,Kerrinlinde,1
1216,CaritasDanmark,CaritasEuropa,1


In [7]:
# for all retweets
graph_weight = nx.from_pandas_edgelist(edgelist_df, "actor", "retweet", edge_attr='weight',create_using=nx.DiGraph())

In [8]:
print(nx.info(graph_weight))

Name: 
Type: DiGraph
Number of nodes: 915
Number of edges: 1218
Average in degree:   1.3311
Average out degree:   1.3311


In [9]:
#export to gephi --> Uncomment 
#nx.write_gexf(graph_weight,'SNA_retweet_all.gexf')

## Centrality scores for all retweets network

In [127]:
retweet_measures = network_measure(edgelist_df)
retweet_measures.sort_values(by=['in_degree'], ascending=False).head(10)

,actor,in_degree,out_degree,closeness,betweeness
452,RasmusPrehn,13,0,0.426903,0.057325
141,DanishMFA,10,0,0.396357,0.056517
721,globaltfokus,9,44,0.428504,0.118088
896,udviklingsmin,8,0,0.366186,0.020755
848,redbarnetdk,7,66,0.446071,0.210471
688,dr2deadline,7,0,0.365746,0.009675
39,AndersLadekarl,7,0,0.350595,0.009117
134,DRC_dk,6,42,0.407854,0.088110
149,Denmark_UN,6,0,0.355504,0.013991
840,politiken,6,0,0.379568,0.011202


# SNA - Small retweet network - Danish humanitarian NGO's

In [10]:
#if receiver is in actor list 
#add to subset of data 

scraped_actors = data_retweets.actor.unique()
small_retweets = pd.DataFrame()

for row in data_retweets.iloc:
    if row.retweet in scraped_actors:
        small_retweets = small_retweets.append(row)

In [11]:
#checking retweets 
i = 0 
for actor in data_retweets.retweet:
    if actor in scraped_actors:
        i +=1
        #print(actor)
print(i)        

121


In [12]:
small_retweets.shape

(121, 2)

In [13]:
retweet_dict = {} 

#for actor, retweet in zip(data_retweets.actor, data_retweets.retweet):
for pair in zip(small_retweets.actor, small_retweets.retweet):
    
    if pair not in retweet_dict:
        retweet_dict[pair] = 1
    else:
        retweet_dict[pair] += 1
        
small_df = pd.DataFrame(columns=["actor", "retweet", "weight"])

actors = []
retweets = []
weights = []

for actor, retweet in retweet_dict:
    weights.append(retweet_dict[(actor, retweet)])
    actors.append(actor)
    retweets.append(retweet)
    
small_df['actor'], small_df['retweet'], small_df['weight'] = actors, retweets, weights

small_df


,actor,retweet,weight
0,PlanBornefonden,globaltfokus,2
1,PlanBornefonden,msf_dk,1
2,PlanBornefonden,oxfamibis,1
3,PlanBornefonden,redbarnetdk,1
4,CARE_Danmark,globaltfokus,2
...,...,...,...
64,AVestegnen,amnestydk,7
65,AVestegnen,redbarnetdk,2
66,AVestegnen,AVestegnen,1
67,AVestegnen,BornsVilkar,1


In [14]:
# for all retweets
graph_weight = nx.from_pandas_edgelist(small_df, "actor", "retweet", edge_attr='weight',create_using=nx.DiGraph())
print(nx.info(graph_weight))
#export to gephi
#uncomment 
#nx.write_gexf(graph_weight,'SNA_retweet_small.gexf')

Name: 
Type: DiGraph
Number of nodes: 24
Number of edges: 69
Average in degree:   2.8750
Average out degree:   2.8750


## Centrality scores for small retweet networks (only NGO's) 

In [72]:
#calculating the in_degree #not weighted 
retweet_measures_small = network_measure(small_df)
retweet_measures_small.sort_values(by=['in_degree'], ascending=False)

,actor,in_degree,out_degree,closeness,betweeness
17,globaltfokus,9,6,0.547826,0.145210
23,redbarnetdk,7,7,0.639130,0.345925
22,oxfamibis,6,5,0.532609,0.081724
5,DRC_dk,6,6,0.532609,0.158733
19,menneskeret,4,1,0.368729,0.009505
6,DignityDK,4,5,0.467656,0.026812
10,UNDP_Danmark,3,2,0.391304,0.013768
13,amnestydk,3,4,0.479348,0.030190
12,WFP_DK,3,2,0.368729,0.004282
3,CARE_Danmark,3,3,0.445905,0.003463


# SNA - Friends network

## Importing data

In [146]:
#Here we start by uploading our twitter file which have all the friends of our actors in it.
friends = pd.read_csv("friends_df1.csv")
friends = friends.iloc[: , 1:] #here remove the first column (which where just the index)

#As friends are the actors friends, which means that the names are users which the actors follows. 
#This means that here, the actors will be the "sender" and the name will be the "reciever", as the relation goes from actors to name.
friends.rename(columns = {'actor' : 'actor', 'name' : 'reciever'}, inplace = True)

friends = friends[['actor', 'reciever']]
friends #here we see the data

,actor,reciever
0,@PlanBornefonden,Andreas Schulz
1,@PlanBornefonden,World Water Week 2021
2,@PlanBornefonden,Yurdal Cicek
3,@PlanBornefonden,Andrea
4,@PlanBornefonden,Sif Klemensen
...,...,...
38189,@ADRA_Danmark,Mette Ditmer Denmark
38190,@ADRA_Danmark,YouGov
38191,@ADRA_Danmark,Natasha Friis Saxberg
38192,@ADRA_Danmark,Marianne S. Delkus


## Preprocessing friends

In [147]:
#here we clean the column of sender, as it does have @ in front.

#first we make two list
cleaned_actor = [] 

for text in friends['actor']:
    item = re.sub(r'@', "", str(text)) #removing @ from sender
    cleaned_actor.append(item)
    
friends['actor'] = cleaned_actor

#dropping NaNs
friends.dropna()

,actor,reciever
0,PlanBornefonden,Andreas Schulz
1,PlanBornefonden,World Water Week 2021
2,PlanBornefonden,Yurdal Cicek
3,PlanBornefonden,Andrea
4,PlanBornefonden,Sif Klemensen
...,...,...
38189,ADRA_Danmark,Mette Ditmer Denmark
38190,ADRA_Danmark,YouGov
38191,ADRA_Danmark,Natasha Friis Saxberg
38192,ADRA_Danmark,Marianne S. Delkus


## Creating friends network

In [148]:
graph_friends = nx.from_pandas_edgelist(friends, 'actor', 'reciever', create_using=nx.MultiDiGraph())

#export to gephi
#uncomment 
#nx.write_gexf(graph_friends,'friends_network.gexf')

print(nx.info(graph_friends))

Name: 
Type: MultiDiGraph
Number of nodes: 1492
Number of edges: 38194
Average in degree:  25.5992
Average out degree:  25.5992


## Centrality scores for friends networks

In [126]:
#calculating the in_degree #not weighted 
friends_measures = network_measure(friends)
friends_measures.sort_values(by=['out_degree'], ascending=False).head(10)

,actor,in_degree,out_degree,closeness,betweeness
193,CARE_Danmark,0,1466,0.983509,0.037182
1477,oxfamibis,0,1466,0.983509,0.037182
1144,SOS_Bornebyerne,0,1466,0.983509,0.037182
172,BornsVilkar,0,1466,0.983509,0.037182
1458,blaakorsdanmark,0,1466,0.983509,0.037182
1459,danmissiondk,0,1466,0.983509,0.037182
1460,danskrodekors,0,1466,0.983509,0.037182
1361,UNDP_Danmark,0,1466,0.983509,0.037182
417,Feministisk_DK,0,1466,0.983509,0.037182
1083,RTC_DK,0,1466,0.983509,0.037182


# SNA - Followers network 

## Importing data

In [149]:
#Here we start by uploading our twitter file which have all the followers in it. 
followers = pd.read_csv("followers_df1.csv")
followers = followers.iloc[: , 1:] #here remove the first column (which where just the index)

#As followers are which users that are following our actor, the name is those who follow our actor. 
#This means that here, the name will be the "sender" and the actors will be the "reciever", as the relation goes from name to actor.
followers.rename(columns = {'actor' : 'reciever', 'name' : 'actor'}, inplace = True)

followers = followers[['actor', 'reciever']]
followers

,actor,reciever
0,Andreas Schulz,@PlanBornefonden
1,Victoria Ellen Gibson,@PlanBornefonden
2,DemwazelWorld,@PlanBornefonden
3,Jeppe Dillermand,@PlanBornefonden
4,Sandra Schmidt,@PlanBornefonden
...,...,...
91580,ADRA Deutschland,@ADRA_Danmark
91581,Impacto Social BR,@ADRA_Danmark
91582,ADRA Denmark,@ADRA_Danmark
91583,ADRA Norway,@ADRA_Danmark


In [150]:
#here we clean the column of sender, as it does have @ in front.

#first we make two list
cleaned_reciever = []

for text in followers['reciever']:
    item = re.sub(r'@', "", str(text)) #removing @ from reciever
    cleaned_reciever.append(item)
    
followers['reciever'] = cleaned_reciever

#dropping NaNs
followers.dropna()

,actor,reciever
0,Andreas Schulz,PlanBornefonden
1,Victoria Ellen Gibson,PlanBornefonden
2,DemwazelWorld,PlanBornefonden
3,Jeppe Dillermand,PlanBornefonden
4,Sandra Schmidt,PlanBornefonden
...,...,...
91580,ADRA Deutschland,ADRA_Danmark
91581,Impacto Social BR,ADRA_Danmark
91582,ADRA Denmark,ADRA_Danmark
91583,ADRA Norway,ADRA_Danmark


## Creating followers network

In [151]:
graph_followers = nx.from_pandas_edgelist(followers, 'actor', 'reciever', create_using=nx.MultiDiGraph())

#export to gephi
#uncomment 
#nx.write_gexf(graph_followers,'followers_network.gexf')

print(nx.info(graph_followers))

Name: 
Type: MultiDiGraph
Number of nodes: 40878
Number of edges: 91585
Average in degree:   2.2404
Average out degree:   2.2404


## Centrality scores for followers networks

In [121]:
in_degree = dict(graph_followers.in_degree())
df_in_degree = pd.DataFrame(list(in_degree.items()),
                                columns = ['actor','in_degree']).sort_values(by='actor', ascending=True)
#out-degree
out_degree = dict(graph_followers.out_degree())
df_out_degree = pd.DataFrame(list(out_degree.items()),
                                 columns = ['actor','out_degree']).sort_values(by='actor', ascending=True)

dfs = [df_in_degree, df_out_degree]
df_network_measures = reduce(lambda left,right: pd.merge(left,right,on='actor'), dfs)

In [143]:
df_network_measures.sort_values(by=['in_degree'], ascending=False).head(10)

,actor,in_degree,out_degree
37261,danskrodekors,9420,0
39039,noedhjaelp,8740,0
39265,redbarnetdk,7585,0
36743,amnestydk,7233,0
7643,DRC_dk,6693,0
38745,menneskeret,5642,0
38910,msf_dk,5520,0
743,ActionAidDK,4780,0
39120,oxfamibis,4690,0
5072,BornsVilkar,4588,0


# VNA - Collaborations 

## Importing immersion journal observations

In [133]:
raw_data = pd.read_excel('VNA.xlsx')
raw_data

,Actor,Receiver,Platform
0,Plan Børnefonden,Danmarks Indsamling,Facebook
1,Plan Børnefonden,Red Barnet,Twitter
2,Plan Børnefonden,Red Barnet,Twitter
3,Plan Børnefonden,Oxfam IBIS,Twitter
4,Plan Børnefonden,WFP Danmark,Twitter
...,...,...,...
308,Menneskeret,Amnesty,Twitter
309,Menneskeret,Amnesty,Twitter
310,Menneskeret,Amnesty,Facebook
311,Menneskeret,"DIGNITY, Danmission",Facebook


In [134]:
df = raw_data.set_index(raw_data.columns.drop('Receiver',1).tolist()).Receiver.str.split(', ', expand=True).stack().reset_index().rename(columns={0:'Receiver'}).loc[:, raw_data.columns]
df = df.rename(columns={'Actor' : 'actor'})
df

,actor,Receiver,Platform
0,Plan Børnefonden,Danmarks Indsamling,Facebook
1,Plan Børnefonden,Red Barnet,Twitter
2,Plan Børnefonden,Red Barnet,Twitter
3,Plan Børnefonden,Oxfam IBIS,Twitter
4,Plan Børnefonden,WFP Danmark,Twitter
...,...,...,...
530,Menneskeret,Amnesty,Twitter
531,Menneskeret,Amnesty,Facebook
532,Menneskeret,DIGNITY,Facebook
533,Menneskeret,Danmission,Facebook


In [144]:
graph_VNA = nx.from_pandas_edgelist(df, 'actor', 'Receiver', edge_attr = 'Platform', create_using=nx.MultiDiGraph())
print(nx.info(graph_VNA))

Name: 
Type: MultiDiGraph
Number of nodes: 32
Number of edges: 535
Average in degree:  16.7188
Average out degree:  16.7188


In [136]:
df_VNA_measures = network_measure(df)
df_VNA_measures.sort_values(by=['in_degree'], ascending=False)

,actor,in_degree,out_degree,closeness,betweeness
23,Red Barnet,14,15,0.688889,0.160234
2,Amnesty,12,4,0.607843,0.138711
8,Danmarks Indsamling,12,0,0.574074,0.089635
18,Mellemfolkeligt Samvirke,9,13,0.645833,0.089151
26,Røde Kors,9,5,0.574074,0.130326
22,Plan Børnefonden,8,4,0.563636,0.099824
21,Oxfam IBIS,8,10,0.596154,0.062281
7,DIGNITY,7,9,0.574074,0.032804
10,Dansk Flygtningehjælp,7,5,0.553571,0.045063
12,Folkekirkens Nødhjælp,6,5,0.563636,0.009478


In [145]:
#export to gephi
#uncomment 
#nx.write_gexf(graph_VNA,'VNA.gexf')